# On Multiple Linear Regression - Codealong

*Note: `scikit-learn` version 0.21.2 contains a necessary revision to [use the `drop` parameter within `OneHotEncoder`](https://scikit-learn.org/stable/whats_new.html#sklearn-preprocessing). Please reinstall your version of `scikit-learn` with the latest version (as of July 5, 2019)*

In [ ]:
!pip install 'scikit-learn==0.21.2' --force-reinstall

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

## Dealing with Categorical Variables

One issue we'd like to resolve is what to do with categorical variables, i.e. variables that represent categories rather than continua. In a Pandas DataFrame, these columns may well have strings or objects for values, but they need not. Recall e.g. the heart-disease dataset from Kaggle in which the target variable took values 0-4, each representing a different stage of heart disease.

### Dummying

One very effective way of dealing with categorical variables is to dummy them out. What this involves is making a new column for _each categorical value in the column we're dummying out_. We'll do this below in our air safety dataset where we have a column of airline names.

These new columns will be filled only with 0's and 1's, a 1 representing the presence of the relevant categorical value.

Let's look at a simple example:

In [ ]:
instructor_data = [{"name": "Brian", "home_state": "CT"},
                   {"name": "Miles", "home_state": "WA"},
                   {"name": "Cristian", "home_state": "IL"},
                   {"name": "Kena", "home_state": "TX"}]

In [ ]:
instructor_df = pd.DataFrame(instructor_data)
instructor_df

> `OneHotEncoder` transforms each categorical feature with `n_categories` possible values into `n_categories` binary features, with one of them 1, and all others 0. - [`sklearn.preprocessing` user guide](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)

In [ ]:
categorical = instructor_df["home_state"].values.reshape(-1, 1)

encoder = OneHotEncoder(drop="first", categories="auto")

encoder.fit(categorical)

These are the categories we fed into the encoder object.

In [ ]:
encoder.categories_

In [ ]:
print(f"""
Without a prefix, the feature names look like this: 
{encoder.get_feature_names()}
      
*******

With a prefix, they look like this: 
{encoder.get_feature_names(['home_state'])}
""")

In [ ]:
ohe = pd.DataFrame(encoder.transform(categorical).toarray(),
                   columns=encoder.get_feature_names(["home_state"]))

ohe.head()

Where did the `home_state_CT` feature go?

## Drug Use Dataset

In [ ]:
drugs = pd.read_csv('raw_data/drug-use-by-age.csv')

In [ ]:
drugs.head()

In [ ]:
drugs.info()

In [ ]:
drugs['age'] = drugs['age'].map(int)

What happened?

In [ ]:
# Let's take a closer look at this 'age' column:

drugs['age'][:15]

In [ ]:
drugs = drugs.head(10)

## Model Selection

Let's imagine that I'm going to try to predict age based on factors to do with drug use.

Now: Which columns (predictors) should I choose? Even ignoring the non-numeric categories in my dataset, there are still 20 predictors I could choose! For each of these predictors, I could either use it or not use it in my model, which means that there are 2^20 = 1,048,576 different models I could construct! Well, okay, one of these is the "empty model" with no predictors in it. But there are still 1,048,575 models from which I can choose!

How can I decide which predictors to use in my model?

### Correlation

In [ ]:
# Use the .corr() DataFrame method to find out about the
# correlation values between all pairs of variables!

drugs.corr()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(8, 8)})

# Use the .heatmap method to depict the relationships visually!
sns.heatmap(drugs.corr());

In [ ]:
# Let's look at the correlations with 'age'
# (our dependent variable) in particular.

drugs.corr()['age'].sort_values(ascending=False)

In [ ]:
X = drugs[['alcohol-use', 'tranquilizer-frequency', 'stimulant-use']]
y = drugs['age']

### Multicollinearity

Probably 'alcohol-use' and 'alcohol-frequency' are highly correlated _with each other_ as well as with 'age'. This can lead to the production of an _overfit_ model. We'll stick a pin in this and return to the issue of overfit models soon.

## Multiple Regression in StatsModels

In [ ]:
import statsmodels.api as sm

In [ ]:
predictors = np.asarray(X)
predictors_int = sm.add_constant(predictors)
model = sm.OLS(y, predictors_int).fit()
model.summary()

In [ ]:
predictors = np.asarray(X2)
predictors_int = sm.add_constant(X2)
model = sm.OLS(np.asarray(y), predictors_int).fit()
model.summary()

## Multiple Regression in Scikit-Learn

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [ ]:
np.random.seed(33)

# Now let's split our data into train and test sets.

X_train, X_test, Y_train, Y_test = train_test_split(X, y)

In [ ]:
# Let's create a StandardScaler object to scale our data for us.
ss = StandardScaler()


# Now we'll apply it to our data by using the .fit_transform() method.

ss.fit(X_train)

X_train_scaled = ss.transform(X_train)

In [ ]:
# Now we can fit the LinearRegression object to our training data!

lr = LinearRegression()
lr.fit(X_train_scaled, Y_train)

In [ ]:
# And score it on our testing set

lr.score(X_test, Y_test)

In [ ]:
# We can use the .coef_ attribute to recover the results
# of the regression.

lr.coef_

## Recursive Feature Elimination

The idea behind recursive feature elimination is to build up (or down) to a small set of predictive features slowly, by eliminating the features with the lowest coefficients.

That is:
1. Start with a model with _all_ $n$ predictors;
2. find the predictor with the smallest coefficient;
3. throw that predictor out and build a model with the remining $n-1$ predictors;
4. set $n = n-1$ and repeat until $n-1$ has the value you want!

### Recursive Feature Elimination in Scikit-Learn

In [ ]:
from sklearn.feature_selection import RFE

lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=2)
select = select.fit(X = drugs.drop(columns=['age',
                                       'cocaine-frequency',
                                       'crack-frequency',
                                       'heroin-frequency',
                                       'inhalant-frequency',
                                       'oxycontin-frequency',
                                       'meth-frequency']),
                    y = drugs['age'])

select.ranking_

In [ ]:
X2 = drugs[['meth-use', 'stimulant-use']]

### Sklearn Metrics

The metrics module in sklearn has a number of metrics that we can use to meaure the accuracy of our model, including the $R^2$ score, the mean absolute error and the mean squared error. Note that the default 'score' on our model object is the $R^2$ score.

In [ ]:
metrics.r2_score(y_test, lr.predict(X_test))

In [ ]:
metrics.mean_absolute_error(y_test, lr.predict(X_test))

In [ ]:
metrics.mean_squared_error(y_test, lr.predict(X_test))